# 3.- Test environment: local prediction inside Docker  

## Quick-test YOLO 11 Segmentation – Docker edition  

This notebook shows how to **run inference with the official YOLO 11 segmentation weights *without* fine-tuning**.  
The workflow is intentionally lightweight: we spin up a Docker container, mount local folders, and call `yolo predict`.

> **Prerequisites**  
> | OS | Requirement | Notes |  
> |----|-------------|-------|  
> | **Windows 10/11** | Docker Desktop | Make sure Docker Desktop is *running* before you execute any cell. |  
> | **Linux (Ubuntu, Fedora, …)** | Docker Engine v20.10 + | You need a user in the `docker` group or run `sudo docker`. |  
> | **macOS (Apple Silicon & Intel)** | Docker Desktop or Colima | If using Apple Silicon, the YOLO image must have an `arm64` tag (the one we build does). |  

All other dependencies (PyYAML, Ultralytics, etc.) live **inside** the Docker image, so your host Python environment stays clean.

We will:

1. Instantiate a Docker client with the Python SDK.  
2. Mount three host directories into the container:  
   * `/data/train` – images you want to test on.  
   * `/data/predict` – output masks & visualisations will be written here.  
   * `/train`     – holds the pre-trained weight file (`yolo11x-seg.pt`).  
3. Execute the YOLO CLI command `yolo predict …` inside the container.  
4. Stream container logs live so you can watch progress and mAP metrics in real time.



In [ ]:
import os
import docker

### 1) Initialise the Docker client

In [22]:
# Create a Docker client
client = docker.from_env()

### 2) Image & volumes


In [23]:
# Docker image name
docker_image = 'yolo11-docker:latest'

In [ ]:
data_local_train_path = os.path.abspath('../data/local_test/train')
data_local_predict_path = os.path.abspath('../data/local_test/predict')
train_local_model_path = os.path.abspath('../train/pretrained_models')

# Map host paths → container paths

volumes = {
    data_local_train_path : {'bind' : '/data/train', 'mode' : 'rw'},
    data_local_predict_path : {'bind' : '/data/predict', 'mode' : 'rw'},
    train_local_model_path  : {'bind' : '/train', 'mode' : 'rw'}
}

### 3) YOLO command (note the CLI uses key=value syntax)


In [ ]:
command = ['yolo', 'predict',
           'model= "/train/yolo11x-seg.pt"',
           'source="/data/train"',
           'project="data"',
           'name="predict"',
           'exist_ok=True']

### 4) Run the container and stream logs

In [26]:
# Run the container and capture the output
try:
    container = client.containers.run(
        docker_image,         # Docker image name
        command,              # Command to execute in container
        volumes=volumes,      # Volumes to mount
        detach=True,          # Run in background (detach mode)
        stdout=True,          # Enable stdout capture
        stderr=True,          # Enable stderr capture
        tty=False             # Disable TTY, since we don’t need interactivity
    )

    print("Container is running...")

    # Collect and print logs in real-time
    for log in container.logs(stream=True):
        print(log.decode('utf-8'), end='')

    # Wait for the container to finish
    container.wait()

    print("Command executed successfully")

except docker.errors.DockerException as e:
    print(f"Error executing Docker command: {e}")

Container is running...
Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
100%|██████████| 119M/119M [00:04<00:00, 29.4MB/s] 
Ultralytics 8.3.99 🚀 Python-3.12.9 torch-2.6.0+cu124 CPU (11th Gen Intel Core(TM) i7-1185G7 3.00GHz)
YOLO11x-seg summary (fused): 203 layers, 62,094,528 parameters, 0 gradients, 319.0 GFLOPs

image 1/3 /data/train/airplane_001.jpg: 640x640 3 airplanes, 1221.3ms
image 2/3 /data/train/bus.jpg: 640x480 4 persons, 1 bus, 1964.2ms
image 3/3 /data/train/panel_2.JPG: 512x640 (no detections), 1822.0ms
Speed: 1.6ms preprocess, 1669.1ms inference, 3.3ms postprocess per image at shape (1, 3, 512, 640)
Results saved to data/predict
💡 Learn more at https://docs.ultralytics.com/modes/predict
Command executed successfull

### What happens next?

* Result masks (`.png`) and annotated images (`labelled.jpg`) are written to `../data/local_test/predict/<run-ID>/`.  
* The CLI also produces a `results.csv` and a `confusion_matrix.png` inside the same folder.  
* You can tweak confidence, IoU, or device selection by adding more `key=value` pairs to the `command` list  
  (e.g., `conf=0.4`, `device=0`, `save_txt=True`).

If you encounter a `platform error` on Apple Silicon, rebuild the Docker image with the line  
`FROM --platform=linux/arm64 python:3.11-slim` at the top of its Dockerfile.
